
# Bu soruya cevap arıyoruz: **γ (gamma) ve β (beta)** neden var? (weight/bias ile ilişkisi)

Görüntü işlemede ve genel derin öğrenmede sık gördüğün iki tür “ayarlama” var:

- **Norm katmanlarında:** öğrenilebilir **γ (scale)** ve **β (shift)**
- **Lineer/Conv katmanlarında:** öğrenilebilir **weight (W)** ve **bias (b)**

Bu notebook şunları netleştirir:

1) γ/β tam olarak ne yapar?  
2) Neden “normalize edip sonra tekrar scale/shift” mantıklı?  
3) Hangi problemi çözer?  
4) γ/β yoksa ne bozulur?  
5) γ/β ile W/bias arasındaki ilişki nedir?  
6) CV pratiklerinde (BN/GN/LN, residual, attention) nerede kritikleşir?




---

## 1) En temel tanım: affine dönüşüm

Bir tensöre uygulanan en basit öğrenilebilir ayar:

\[
y = a x + b
\]

- \(a\): **scale** (ölçekleme)
- \(b\): **shift** (kaydırma)

Norm katmanlarındaki γ ve β tam olarak budur:

\[
y = \gamma \hat{x} + \beta
\]

Burada \(\hat{x}\) normalize edilmiş aktivasyondur.



---

## 2) “Normalize et → sonra γ/β ile geri koy” neden var?

Normalizasyon (örn BatchNorm) şu işi yapar:

\[
\hat{x} = \frac{x-\mu}{\sqrt{\sigma^2+\epsilon}}
\]

Bu işlem:
- ölçek farklarını bastırır
- dağılımı stabilize eder
- optimizasyonu kolaylaştırır

Ama tek başına şunu da yapar:
- ağın “istediği” ölçek/ortalama bilgisini **zorla siler**

İşte γ/β burada devreye girer:

- **γ**: ağın istediği **varyansı/ölçeği** geri koymasına izin verir
- **β**: ağın istediği **ortalama/ofseti** geri koymasına izin verir

📌 Kilit fikir:
> Normalizasyon “optimizasyon için” yapılır.  
> γ/β ise “temsil gücü kaybı olmasın” diye eklenir.



---

## 3) Hangi problemi çözer?

### 3.1) Temsil gücü kaybını engeller

Eğer sadece normalize edersen, her kanal/feature zorla:
- ortalama ≈ 0
- std ≈ 1

kalır.

Bu durumda ağın bazı katmanlarda “belli bir ölçek/offset” istemesi mümkün olabilir.
Örneğin:
- ReLU eşiği
- residual branch ölçek dengesi
- attention gating için uygun logit ölçekleri

γ/β **bu özgürlüğü geri verir.**

### 3.2) “Norm koyunca ağ bozulmasın” garantisi (identity mapping kapasitesi)

Çok kritik: BN + γ/β ile ağ isterse:

- \(\gamma=1, \beta=0\) seçerek “standart normalize” davranır
- ya da \(\gamma,\beta\) öğrenerek belirli kanalları büyütüp küçültür

Bazı durumlarda γ/β, norm katmanının “zarar” verme riskini azaltır.



---

## 4) Hangi sorunun olmamasına olanak tanır?

### 4.1) “Normalize ettiğim için bilgi kaybettim” sorunu

Sadece normalize etsen:
- ölçek ve ofset bilgisi “sabitlenmiş” olur
- modelin ifade gücü sınırlanır

γ/β ile:
- normalizasyonun getirdiği stabilite korunur
- ama model “gereken” ölçek/ofseti geri öğrenir

### 4.2) Aktivasyon fonksiyonlarıyla uyum (özellikle ReLU)

ReLU’da negatifler sıfırlanır.
Eğer β ile dağılımı kaydırmazsan bazı kanallar gereksiz yere “ölü” kalabilir.
β, aktivasyonların eşik etrafında daha iyi konumlanmasına yardım eder.

### 4.3) Residual toplama dengesi

Residual blokta:
\[
y = x + F(x)
\]

F(x) dalının ölçeği x’e göre çok büyük/küçük olursa:
- optimizasyon zorlaşır
- eğitim dengesizleşir

Norm içindeki γ, bu dalın ölçeğini ayarlamada kritik rol oynar.



---

## 5) γ/β ile weight/bias (W,b) arasındaki ilişki

### 5.1) Conv/Linear:
\[
y = Wx + b
\]

Burada:
- W: yön/karışım öğrenir (feature mixing)
- b: sabit ofset

### 5.2) Norm + affine:
\[
y = \gamma \hat{x} + \beta
\]

Burada:
- γ: her kanal/feature için ayrı ölçek (diagonal scaling gibi)
- β: her kanal/feature için ofset

Önemli fark:
- W “kanallar arası karışım” yapabilir.
- γ ise genelde “kanal başına” ölçekler (daha sınırlı ama stabil).

📌 Pratik sonuç:
> γ/β, norm katmanını “öğrenilebilir ve zarar vermeyen” hale getirir;  
> W/b ise asıl dönüşümü (feature karışımını) öğrenir.



---

## 6) BN’de bias neden bazen kapatılır?

Sık görülen pratik:

- Conv bias = False
- Sonra BatchNorm gelir

Neden?

BatchNorm zaten β ile ofset eklediği için, conv bias çoğu zaman redundant olur.
Ayrıca BN, conv çıkışını normalize ederken bias etkisini “absorbe” eder.

Bu yüzden:
> Conv bias + BN β aynı işin iki kopyası gibi davranabilir.

(Bu %100 her senaryoda değil ama genel pratik böyle.)


In [1]:

# Basit bir gösterim: BN'nin affine parametreleri conv bias'ı çoğu zaman gereksiz kılar.
# Bu hücre teorik: sayısal örnekle "bias + normalize" etkisini gösterir.

import numpy as np

np.random.seed(0)
x = np.random.randn(8, 1)          # küçük batch
b = 2.0                            # conv bias gibi düşün
x_b = x + b

mu = x_b.mean()
std = x_b.std() + 1e-5
x_hat = (x_b - mu) / std

gamma = 1.0
beta = 0.0
y = gamma * x_hat + beta

print("x mean/std:", x.mean(), x.std())
print("x+b mean/std:", x_b.mean(), x_b.std())
print("normalize(x+b) mean/std:", y.mean(), y.std())


x mean/std: 0.8841065071429416 1.022668337083785
x+b mean/std: 2.884106507142941 1.022668337083785
normalize(x+b) mean/std: 4.718447854656915e-16 0.9999902217543507



---

## 7) CV’de γ/β ne zaman “özellikle” önem kazanır?

### 7.1) Small batch / GN/LN kullanımı
BatchNorm yerine GN/LN kullanırken de affine (γ/β) kritik; çünkü norm “sabit bir standardizasyon” gibi kalmasın.

### 7.2) Attention gating logit ölçeği
Sigmoid/softmax öncesi logit ölçeği çok küçük/büyük olursa saturasyon olur.
γ, bu ölçeği ayarlamaya yardım eder.

### 7.3) Residual branch stabilizasyonu
Bazı mimarilerde γ başlangıçta küçük ayarlanır (ör. bazı residual tasarımlarda son BN γ=0 başlatma) → başlangıçta blok “identity”ye yakın davranır → stabil eğitim.

Bu, “γ sadece ölçek” değil, aynı zamanda **eğitim dinamiği kontrol knob’u** demek.



---

## 8) Kapanış: net cevap

- Normalizasyon, aktivasyonları standardize ederek optimizasyonu kolaylaştırır.
- Ama bu standardizasyon, ağın istediği ölçek/ofseti zorla sabitleyebilir.
- **γ (gamma) ve β (beta)** bu kaybı telafi eder:
  - γ: ölçeği geri koyar (scale)
  - β: ortalamayı geri koyar (shift)
- Böylece norm katmanı:
  - **stabil** (normalize)
  - ama **esnek** (öğrenilebilir affine)
  olur.

Yani γ/β, normalizasyonun “stabilite” faydasını alırken “temsil gücü kaybı” problemini engeller.
